In [16]:
"""
NOTE: Deep learning is a machine learning technique that teaches computers to do what comes naturally 
to humans: learn by example. Deep learning is a key technology behind driverless cars, enabling them 
to recognize a stop sign, or to distinguish a pedestrian from a lamppost.
"""

"""
Today, we will implement our first multi-layer neural network that can do digit classification based
on the famous MNIST dataset

We will put all of the things from the last tutorials together:
    > MNIST Dataset
    > DataLoader, Transformation
    > Multilayer Neural Network (input layer, hidden layer, output layer), Activation functions
    > Loss and Optimizer
    > Training loop (batch training)
    > Model evaluation
    > GPU support
"""

'\nToday, we will implement our first multi-layer neural network that can do digit classification based\non the famous MNIST dataset\n\nWe will put all of the things from the last tutorials together:\n    > MNIST Dataset\n    > DataLoader, Transformation\n    > Multilayer Neural Network (input layer, hidden layer, output layer), Activation functions\n    > Loss and Optimizer\n    > Training loop (batch training)\n    > Model evaluation\n    > GPU support\n'

In [17]:
import torch
import torch.nn as nn
import torchvision # For datasets
import torchvision.transforms as transforms # For transforms
import matplotlib.pyplot as plt # To graph data

In [18]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# cuda = gpu, cpu = cpu

print(device)

cpu


In [19]:
# Hyper parameters

input_size = 784 # The images are 28x28 which when flattened is 1x784
hidden_size = 100 # ??? Can try out different sizes here
num_classes = 10 # 10 classes - digits form 0 to 9
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [20]:
# Import MNIST dataset

# Dataset for training
train_dataset = torchvision.datasets.MNIST(root = './data', 
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True)
# root for root folder
# train = True means it is a training dataset
# transform ToTensor makes the data a tensor
# download = True means it should be downloaded if it is not already

# Dataset for testing
test_dataset = torchvision.datasets.MNIST(root = './data', 
                                           train = False,
                                           transform = transforms.ToTensor())
# Notice, train = False and no download needed


# Data loaders

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle = True)
# shuffle = True shuffles the data around - good for testing

test_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle = False)
# Shuffle does not matter for the evaluation

# Check if it all works
# I can't get iter to work

# Samples output is [100, 1, 28, 28]
# First one because 100 samples in our batch
# No color channels so next one is just 1
# Next 2 are image i.e. 28 x 28

# Labels is just a size 100 tensor (just 100 labels for which number it is)


# Plot it
"""
for i in range(6):
    plt.subplot (2, 3, i + 1) # 2 rows, 3 columns, index = i + 1
    plt.imshow(samples[i])
"""

# The above shows images

'\nfor i in range(6):\n    plt.subplot (2, 3, i + 1) # 2 rows, 3 columns, index = i + 1\n    plt.imshow(samples[i])\n'

In [21]:
"""
We now want to classify these images (hadnwritten numbers as digits 0-9), we will set up a fully
connected neural network with one hidden layer
"""
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        # Needs to have input size, hidden size and ouput size (which is the number of classes)
        super(NeuralNet, self).__init__()
        
        # Create our layers
        self.l1 = nn.Linear(input_size, hidden_size) # nn.Linear(inputsize, outputsize)
        self.relu = nn.ReLU() # ReLu activation function
        self.l2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        
        # We don't apply softmax because this is a multi-class problem
        # We will use cross-entropy loss (kinda applies softmax for us)
        # Instead, we just return "out"
        
        return out
        
model = NeuralNet(input_size, hidden_size, num_classes)

In [22]:
# Create Loss and optimizer

# Loss funciton
criterion = nn.CrossEntropyLoss() # Applies softmax for us so we don't put it in the model

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # What is this optimizer

In [25]:
# Training loop

n_total_steps = len(train_loader)

running_loss = 0.0
running_correct = 0 # Running correct predictions

# First, loop over epochs
for epoch in range(num_epochs):
    # Next, loop over batches
    for i, (images, labels) in enumerate(train_loader): # enumerate gives us the inedex
        ## We have to reshae our images first
        # Shape is currently 100 x 1 x 28 x 28
        # But our input size is (28 * 28=) 784
        # So our image tensor needs shape 100 x 784 (n_bacthes x image_size)
        images = images.reshape(-1, 28*28).to(device) # -1 is found out automatically for us
        labels = labels.to(device)
        # .to(device) pushes it to gpu if possible
        
        ## Forward pass
        outputs = model(images)
        # Calculate loss
        loss = criterion(outputs, labels)
        
        ## Backwards pass
        # First, empty values in gradient attribute
        optimizer.zero_grad()
        # Do backward propogation
        loss.backward()
        # Update parameters
        optimizer.step()
        
        ## Print loss
        
        # Tensorboard
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        running_correct += (predicted == labels).sum().item()
        
        if ((i+1) % 100 == 0): # Every 100th statement
            print(f'epoch{epoch+1}/{num_epochs}, step{i+1}/{n_total_steps}, loss = {loss.item():.4f}')
            
            writer.add_scalar('training_loss', running_loss/100, epoch * n_total_steps + i) # Last one is current global step
            writer.add_scalar('accuracy', running_correct/100, epoch * n_total_steps + i)
            running_loss = 0.0
            running_correct = 0
                
                

epoch1/2, step100/600, loss = 0.2782
epoch1/2, step200/600, loss = 0.2487
epoch1/2, step300/600, loss = 0.1877
epoch1/2, step400/600, loss = 0.2146
epoch1/2, step500/600, loss = 0.1574
epoch1/2, step600/600, loss = 0.2355
epoch2/2, step100/600, loss = 0.3102
epoch2/2, step200/600, loss = 0.1732
epoch2/2, step300/600, loss = 0.2400
epoch2/2, step400/600, loss = 0.1188
epoch2/2, step500/600, loss = 0.1559
epoch2/2, step600/600, loss = 0.2153


In [ ]:
## Testing

# We don't want to comput the gradients

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    
    # Loop over all batches in test samples
    for images, labels in test_loader:
        # Reshape this as we did above
        images = images.reshape(-1, 28*28).to(device) # -1 is found out automatically for us
        labels = labels.to(device)
        
        # Calculate predictions
        outputs = model(images) # model() is trained now and is given test images
        _, predictions = torch.max(outputs, 1)
        # torch.max returns value, index (index = class label)
        n_samples += labels.shape[0] # Gives us number of samples in current batch (should be 100)
        n_correct += (predictions == labels).sum().item() # Fancy way of doing "for each prediction
        # ... if it is right add 1 to n_corrrect
        
    # Total accuracy
    accuracy = 100 * (n_correct/n_samples)
    print(f'accuracy = {accuracy}') # 95.5%

In [ ]:
"""
Important part I keep forgetting

for epoch in range(num_epochs): (1)
    for i, (images, labels) in enumerate(train_loader): (2)
    
(1) loops over the EPOCHS (how many times we are training over the data)
(2) loops over the batches of each epoch (like the parts of each epoch)
"""

In [ ]:
# 15 - Tensorboard

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/mnist') # Set up writer

examples = iter(test_loader)
example_data, example_targets = next(examples)

# For showing images

img_grid = torchvision.utils.make_grid(example_data)
writer.add_image('mnist_images', img_grid)

# Go into VizDoomProject and do "tensorboard --logdir=runs"


# For graph
                                       
writer.add_graph(model, example_data.reshape(-1, 28*28))
writer.close()


# For metrics

